**1. Exploratory Data Analysis (EDA)**: 

   - Feature engineering (if needed): change or add new x's: x1 -> x1^2
     
   - Feature selection: eg. 100 features， 选 15 个最有用的features

## <center> Overview </center>

| *Visualization types* | continuous | categorical |
| --- | :-: | :-: |
| __continuous__    	| Hexbin Plot, Contour Plot, 2D Histogram, Pair Plot, Line Plot 	| Strip Plot, Swarm Plot, Point Plot, Bar Plot, Facet Plot|
| __categorical__   	| Strip Plot, Swarm Plot, Point Plot, Bar Plot, Facet Plot |  Grouped Bar Plot, Mosaic Plot, Heatmap, Chi-Square Plot, Alluvial Plot  	|

## <center> Overview </center>

| *Visualization types* |  continuous |  categorical |
| --- | :-: | :-: |
| __continuous__    	| scatter plot, heatmap 	| category-specific histograms, box plot, violin plot |
| __categorical__   	| category-specific histograms, box plot, violin plot |  stacked bar plot  	|


**2. Split the data into different sets**: most often the sets are train, validation, and test



- Feature engineering

   - (SMALL features and LARGE points)
   - combine features in a simple and automatic way (PolynomialFeatures method in sklearn)
   - if n_ftrs << n_points, this can modestly improve the predictive power of your model 

In [1]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures


X = np.arange(6).reshape(3, 2)
print(X)

poly = PolynomialFeatures(2)
print(poly.fit_transform(X)) # [1, a, b, a^2, ab, b^2]
poly = PolynomialFeatures(2, include_bias=False) # bias = 1
print(poly.fit_transform(X)) # [a, b, a^2, ab, b^2]
poly = PolynomialFeatures(2,interaction_only=True, include_bias=False) #interaction: a, b, ab
print(poly.fit_transform(X)) # [a, b, ab]

[[0 1]
 [2 3]
 [4 5]]
[[ 1.  0.  1.  0.  0.  1.]
 [ 1.  2.  3.  4.  6.  9.]
 [ 1.  4.  5. 16. 20. 25.]]
[[ 0.  1.  0.  0.  1.]
 [ 2.  3.  4.  6.  9.]
 [ 4.  5. 16. 20. 25.]]
[[ 0.  1.  0.]
 [ 2.  3.  6.]
 [ 4.  5. 20.]]


 - Feature selection
    -  (LARGE features and SMALL points)
    - you have too many features: n_ftrs > n_points (some algorithms break down)
    - if training an ML algorithm is too computationally expensive using all the features

## The steps

**1. Exploratory Data Analysis (EDA)**: you need to understand your data and verify that it doesn't contain errors
   - do as much EDA as you can!


   - Feature engineering (if needed): change or add new x's: x1 -> x1^2
     
   - Feature selection: eg. 100 features， 选 15 个最有用的features
      

**2. Split the data into different sets**: most often the sets are train, validation, and test (or holdout)
   - practitioners often make errors in this step!
   - you can split the data randomly, based on groups, based on time, or any other non-standard way if necessary to answer your ML question

**3. Preprocess the data**: ML models only work if X and Y are numbers! Some ML models additionally require each feature to have 0 mean and 1 standard deviation (standardized features)
   - often the original features you get contain strings (for example a gender feature would contain 'male', 'female', 'non-binary', 'unknown') which needs to be transformed into numbers
   - often the features are not standardized (e.g., age is between 0 and 100) but it needs to be standardized

   - **OneHotEncoder** - converts categorical features into dummy arrays
   - **OrdinalEncoder** - converts ordinal features into an integer array
   - **MinMaxScaler** - scales continuous variables to be between 0 and 1
   - **StandardScaler** - standardizes continuous features by removing the mean and scaling to unit variance

   - **Missing values:**
     - Missing values in ordinal and categorical features can be automatically dealt by `OneHotEncoder` and `OrdinalEncoder`.  Replace the NaN with a string first
     - The missing values in continuous features can be done in one of the following techniques:
       - apply multivariate imputation `SimpleImputer`
       - apply XGBoost model to a dataset with missing values
       - apply the reduced-features model (also called the pattern sub-model approach)


    
**4. Choose an evaluation metric**: depends on the priorities of the stakeholders
   -  Classification evaluation metrics: confusion matrix, accuracy, recall, precision, f-score, ROC, precision-recall curves, log-loss metric
      - ROC AUC - ROC Area Under the Curve (算面积 得到一个数值)
        - AUC = 1 is a perfect classifier
        - AUC > 0.5 is above chance-level predictor
        - AUC = 0.5 is a chance-level classifier
        - AUC < 0.5 is a bad predictor
        - AUC = 0 classifies all points incorrectly
   -  Regression evaluation metrics: MSE, RMSE, MAE, R^2

   - the model performs very good on the training set but poorly on the validation set when all features are used
   - ## Regularization to the rescue!
   - let's change the cost function and add a <font color='RED'>penalty term</font> for large ws
   - **Lasso regression**: regularize using the l1 norm of w:
      
$L(w) = \frac{1}{n}\sum_{i=1}^{n}[(w_0 + \sum_{j=1}^{m} w_j  x_{ij}- y_i)^2] + \color{red}{ \alpha \sum_{j=0}^{m}|w_j|}$ 
      
   - **Ridge regression**: regularize using the square of the l2 norm of w:
      
$L(w) = \frac{1}{n}\sum_{i=1}^{n}[(w_0 + \sum_{j=1}^{m} w_j  x_{ij}- y_i)^2] + \color{red}{\alpha \sum_{j=0}^{m} w_j^2}$

   - $\alpha$ is the regularization parameter (positive number), it describes how much we penalize large ws

   - With the cost function changed, the derivatives in gradient descent need to be updated too!   


  
  <table>
    <tr>
        <td colspan="2" rowspan="2"></td>
        <td colspan="2">Predicted class</td>			
    </tr>
    <tr>
        <td>Predicted Negative (0)</td>
        <td>Predicted Positive (1)</td>
    </tr>
    <tr>
        <td rowspan="2">Actual class</td>
        <td>Condition Negative (0)</td>
        <td><b>True Negative (TN)</b></td>
        <td><b>False Positive (FP)</b></td>
    </tr>
    <tr>
        <td>Condition Positive (1)</td>
        <td><b>False Negative (FN)</b></td>
        <td><b>True Positive (TP)</b></td>
    </tr>
</table>

  - accuracy: fraction of data points correctly classified
    - a = (TP + TN) / (TP + TN + FP + FN)
  - recall: what fraction of the condition positive samples are true positives?
    - it measures the ability of the classifier to identify all positive samples
    - in binary classification: R = TP / (TP + FN) #想要避免FN FN is expensive. eg: 实际有病但没检测出来
  - precision: what fraction of the predicted positive points are true positives?
    - it measures the ability of the classifier to not predict a negative sample to be positive
    - in binary classification: P = TP / (TP + FP)
  - The f_beta score (imbalanced data)
  - 
    Weighted harmonic mean of P and R:
    ###  $f_{\beta} = (1 + \beta^2) \frac{P R}{\beta^2 P + R}$ 

    If $\beta = 1$, we have the f1 score:
    ###  $f_{1} = 2 \frac{P R}{P + R}$ 

    If $\beta < 1$, more weight to precision. Expensive to act. 

    If $\beta > 1$, more weight to recall. Cheap to act. 
     
**5. Choose one or more ML techniques**: it is highly recommended that you try multiple models
   - start with simple models like linear or logistic regression
   - try also more complex models like nearest neighbors, support vector machines, random forest, etc.

   - linear regression: continuous target
   - logistic regression: classification target


| ML algo | suitable for large datasets? | behavior with respect to outliers | non-linear? | params to tune | smooth predictions | easy to interpret? |
| - | :-: | :-: | :-: | :-: | :-: | :-: |
| linear regression            	|              yes             	|linear extrapolation|      no     	| l1 and/or l2 reg 	| yes | yes|
| logistic regression          	|              yes             	|scales with distance from the decision boundary|      no     	| l1 and/or l2 reg 	| yes | yes|
| random forest regression     	|so so |constant|yes|max_features,  max_depth| no|so so|
| random forest classification 	|so so |step-like, difficult to tell|yes|max_features,  max_depth| no|so so|
| SVM rbf regression               	|no|non-linear extrapolation|yes|C, gamma|yes|so so|
| SVM rbf classification           	|no| fifty-fifty | yes| C, gamma| yes| so so |
    
**6. Tune the hyperparameters of your ML models (aka cross-validation)**
   - regularization if over fitting ()
   - ML techniques have hyperparameters that you need to optimize to achieve best performance
   - for each ML model, decide which parameters to tune and what values to try
   - loop through each parameter combination
       - train one model for each parameter combination
       - evaluate how well the model performs on the validation set
   - take the parameter combo that gives the best validation score
   - evaluate that model on the test set to report how well the model is expected to perform on previously unseen data
    
**7. Interpret your model**: black boxes are often not useful
   - check if your model uses features that make sense (excellent tool for debugging)
   - often model predictions are not enough, you need to be able to explain how the model arrived to a particular prediction (e.g., in health care)
   - Global feature importance metrics: 
     - Permutation feature importance 
     - coefficients of linear models
   - Local feature importance metrics: 
     - `Shap` values
     - Locally Interpretable Model-agnostic Explanations

SVC 
LogisticRegression 
XGBoost 
RandomForestClassifier 

For imbalanced classification. 

In [ ]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

#np.random.seed(0)

df = pd.read_csv('data/adult_data.csv')

# let's separate the feature matrix X, and target variable y
y = df['gross-income'] # remember, we want to predict who earns more than 50k or less than 50k
X = df.loc[:, df.columns != 'gross-income'] # all other columns are features

random_state = 42

# first split to separate out the training set
X_train, X_other, y_train, y_other = train_test_split(X,y,train_size = 0.6,random_state=random_state)

# second split to separate out the validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,random_state=random_state)


In [ ]:
# collect which encoder to use on each feature
# target 永远不preprocess
# needs to be done manually
ordinal_ftrs = ['education'] 
ordinal_cats = [[' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',' 12th',' HS-grad',\
                ' Some-college',' Assoc-voc',' Assoc-acdm',' Bachelors',' Masters',' Prof-school',' Doctorate']]
onehot_ftrs = ['workclass','marital-status','occupation','relationship','race','sex','native-country']
minmax_ftrs = ['age','hours-per-week']
std_ftrs = ['capital-gain','capital-loss']

# collect all the encoders
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories = ordinal_cats), ordinal_ftrs),
        ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'), onehot_ftrs),
        ('minmax', MinMaxScaler(), minmax_ftrs),
        ('std', StandardScaler(), std_ftrs)])


clf = Pipeline(steps=[('preprocessor', preprocessor)]) # for now we only preprocess 
                                                       # later on we will add other steps here

X_train_prep = clf.fit_transform(X_train)
X_val_prep = clf.transform(X_val)
X_test_prep = clf.transform(X_test)

print(X_train.shape)
print(X_train_prep.shape)
print(X_train_prep)


1. baseline_accuracy
2. baseline in SHAP
3. train cv test?
4. 

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, KFold, GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import make_scorer, fbeta_score
# import joblib


# def MLpipe_KFold_FBeta(X, y, preprocessor, ML_algo, param_grid):
#     """
#     Function to perform KFold cross-validation, hyperparameter tuning with GridSearchCV, and evaluate with F-beta score.
#     Args:
#         X (pd.DataFrame): Feature matrix.
#         y (pd.Series): Target variable.
#         preprocessor (ColumnTransformer): Preprocessor pipeline for data transformation.
#         ML_algo (Estimator): Uninitialized machine learning algorithm (e.g., RandomForestClassifier()).
#         param_grid (dict): Parameter grid for hyperparameter tuning.
#     Returns:
#         test_scores (list): List of 10 test fbeta scores (1 per random state).
#         best_models (list): List of 10 best models (1 per random state).
#     """
#     # Lists to store test scores and best models for each random state
#     test_scores = []
#     best_models = []

#     # Define the fbeta scoring function
#     fbeta_scorer = make_scorer(fbeta_score, beta=1, greater_is_better=True)

#     # Loop through 10 different random states
#     for random_state in range(10):
#         print(f"\n========== Random State: {random_state} ==========")

#         # Split the data into 'other' (train + validation) and test (80/20 split)
#         X_other, X_test, y_other, y_test = train_test_split(
#             X, y, test_size=0.2, stratify=y, random_state=random_state
#         )
#         # print(f"Data split - X_other: {X_other.shape}, X_test: {X_test.shape}")

#         # Apply KFold with 4 splits on the 'other' set
#         kf = KFold(n_splits=4, shuffle=True, random_state=random_state)

#         # Create a pipeline with the preprocessor and ML algorithm
#         pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                                     ('model', ML_algo)])
        
#         # Use GridSearchCV for hyperparameter tuning
#         grid_search = GridSearchCV(
#             estimator=pipeline,
#             param_grid=param_grid,
#             cv=kf,
#             scoring=fbeta_scorer,
#             n_jobs=-1
#         )

#         # Fit the pipeline with GridSearchCV
#         print("Performing GridSearchCV...")
#         grid_search.fit(X_other, y_other)

#         # Print the best parameters found
#         print(f"Best parameters found for Random State {random_state}: {grid_search.best_params_}")

#         # Save the best model from this random state
#         best_model = grid_search.best_estimator_
#         best_models.append(best_model)
        
#         # Save the model using joblib
#         model_filename = f"best_model_state_{random_state}.pkl"
#         joblib.dump(best_model, model_filename)
#         print(f"Saved best model for Random State {random_state} as {model_filename}")

#         # Predict on the test set and calculate fbeta score
#         y_test_pred = best_model.predict(X_test)
#         test_fbeta = fbeta_score(y_test, y_test_pred, beta=1)
#         test_scores.append(test_fbeta)
#         print(f"Test F-beta score for Random State {random_state}: {test_fbeta:.4f}")

#     # Calculate mean and standard deviation of test scores across all random states
#     mean_test_score = np.mean(test_scores)
#     std_test_score = np.std(test_scores)
#     print("\n========== Summary of Test Scores ==========")
#     print(f"Mean Test F-beta Score: {mean_test_score:.4f}")
#     print(f"Standard Deviation of Test Scores: {std_test_score:.4f}")

#     # Return the list of test scores and best models
#     return test_scores, best_models


In [ ]:
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
import numpy as np
import joblib

def MLpipe_KFold_AUC_PR(X, y, preprocessor, ML_algo, param_grid):
    """
    Function to perform KFold cross-validation, hyperparameter tuning with GridSearchCV, and evaluate with AUC-PR.
    Args:
        X (pd.DataFrame): Feature matrix.
        y (pd.Series): Target variable.
        preprocessor (ColumnTransformer): Preprocessor pipeline for data transformation.
        ML_algo (Estimator): Uninitialized machine learning algorithm (e.g., RandomForestClassifier()).
        param_grid (dict): Parameter grid for hyperparameter tuning.
    Returns:
        test_scores (list): List of 10 test AUC-PR scores (1 per random state).
        best_models (list): List of 10 best models (1 per random state).
    """
    # Lists to store test scores and best models for each random state
    test_scores = []
    best_models = []

    # Define the AUC-PR scoring function
    auc_pr_scorer = make_scorer(average_precision_score, needs_proba=True)

    random_states = []
    # Loop through 5 different random states

    for i in range(5):
        random_states.append(42 * i)


    for random_state in random_states:
        print(f"\n========== Random State: {random_state} ==========")

        # Split the data into 'other' (train + validation) and test (80/20 split)
        X_other, X_test, y_other, y_test = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=random_state
        )
        # print(f"Data split - X_other: {X_other.shape}, X_test: {X_test.shape}")

        # Apply KFold with 4 splits on the 'other' set
        kf = KFold(n_splits=4, shuffle=True, random_state=random_state)

        # Create a pipeline with the preprocessor and ML algorithm
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                    ('model', ML_algo)])
        
        # Use GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            cv=kf,
            scoring=auc_pr_scorer,
            n_jobs=-1
        )

        # Fit the pipeline with GridSearchCV
        print("Performing GridSearchCV...")
        grid_search.fit(X_other, y_other)

        # Print the best parameters found
        print(f"Best parameters found for Random State {random_state}: {grid_search.best_params_}")

        # Save the best model from this random state
        best_model = grid_search.best_estimator_
        best_models.append(best_model)
        
        # Save the model using joblib
        model_filename = f"best_model_state_{random_state}.pkl"
        joblib.dump(best_model, model_filename)
        print(f"Saved best model for Random State {random_state} as {model_filename}")

        # Predict probabilities on the test set and calculate AUC-PR
        y_test_prob = best_model.predict_proba(X_test)[:, 1]  # Use probabilities for the positive class
        test_auc_pr = average_precision_score(y_test, y_test_prob)
        test_scores.append(test_auc_pr)
        print(f"Test AUC-PR score for Random State {random_state}: {test_auc_pr:.4f}")

    # Calculate mean and standard deviation of test scores across all random states
    mean_test_score = np.mean(test_scores)
    std_test_score = np.std(test_scores)
    print("\n========== Summary of Test Scores ==========")
    print(f"Mean Test AUC-PR Score: {mean_test_score:.4f}")
    print(f"Standard Deviation of Test AUC-PR Score: {std_test_score:.4f}")

    # Return the list of test scores and best models
    return test_scores, best_models
    

results = {}

# Loop through each model, run the pipeline, and collect results
for model_name, (model, param_grid) in models.items():
    print(f"\n--- Running model: {model_name} ---")
    test_scores, best_models = MLpipe_KFold_AUC_PR(X, y, preprocessor, model, param_grid)
    mean_auc_pr = np.mean(test_scores)
    std_auc_pr = np.std(test_scores)
    results[model_name] = (mean_auc_pr, std_auc_pr)  # Store results in dictionary
    print(f"\nMean Test AUC-PR Score for {model_name}: {mean_auc_pr:.4f}")
    print(f"Standard Deviation of Test AUC-PR Score for {model_name}: {std_auc_pr:.4f}")


In [ ]:
'XGBClassifier': (
        XGBClassifier(eval_metric='logloss'),
        {"model__learning_rate": [0.03],
        "model__n_estimators": [10000],
        "model__seed": [0],
        # "model__reg_alpha": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
        # "model__reg_lambda": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
        # "model__max_depth": [1,3,10,30,100],
        "model__reg_alpha": [1e-1, 1e0, 1e1],
        "model__reg_lambda": [ 1e-1, 1e0, 1e1,],
        "model__max_depth": [1,3,10],
        "model__colsample_bytree": [0.9],              
        "model__subsample": [0.66]}
    )

'SVC': (
    SVC(probability=True),
    {'model__gamma': [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3],
    'model__C': [1e-2, 1e-1, 1e0, 1e1, 1e2]})

In [ ]:
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
import numpy as np
import joblib

def MLpipe_KFold_AUC_PR(X, y, preprocessor, ML_algo, param_grid):
    """
    Function to perform KFold cross-validation, hyperparameter tuning with GridSearchCV, and evaluate with AUC-PR.
    Args:
        X (pd.DataFrame): Feature matrix.
        y (pd.Series): Target variable.
        preprocessor (ColumnTransformer): Preprocessor pipeline for data transformation.
        ML_algo (Estimator): Uninitialized machine learning algorithm (e.g., RandomForestClassifier()).
        param_grid (dict): Parameter grid for hyperparameter tuning.
    Returns:
        test_scores (list): List of 10 test AUC-PR scores (1 per random state).
        best_models (list): List of 10 best models (1 per random state).
    """
    # Lists to store test scores and best models for each random state
    test_scores = []
    best_models = []

    # Define the AUC-PR scoring function
    auc_pr_scorer = make_scorer(average_precision_score, needs_proba=True)

    random_states = []
    # Loop through 5 different random states

    for i in range(5):
        random_states.append(42 * i)


    for random_state in random_states:
        print(f"\n========== Random State: {random_state} ==========")

        # Split the data into 'other' (train + validation) and test (80/20 split)
        X_other, X_test, y_other, y_test = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=random_state
        )
        # print(f"Data split - X_other: {X_other.shape}, X_test: {X_test.shape}")

        # Apply KFold with 4 splits on the 'other' set
        kf = KFold(n_splits=4, shuffle=True, random_state=random_state)

        # Create a pipeline with the preprocessor and ML algorithm
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                    ('model', ML_algo)])
        
        # Use GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            cv=kf,
            scoring=auc_pr_scorer,
            n_jobs=-1
        )

        # Fit the pipeline with GridSearchCV
        print("Performing GridSearchCV...")
        grid_search.fit(X_other, y_other)

        # Print the best parameters found
        print(f"Best parameters found for Random State {random_state}: {grid_search.best_params_}")

        # Save the best model from this random state
        best_model = grid_search.best_estimator_
        best_models.append(best_model)
        
        # Save the model using joblib
        model_filename = f"best_model_state_{random_state}.pkl"
        joblib.dump(best_model, model_filename)
        print(f"Saved best model for Random State {random_state} as {model_filename}")

        # Predict probabilities on the test set and calculate AUC-PR
        y_test_prob = best_model.predict_proba(X_test)[:, 1]  # Use probabilities for the positive class
        test_auc_pr = average_precision_score(y_test, y_test_prob)
        test_scores.append(test_auc_pr)
        print(f"Test AUC-PR score for Random State {random_state}: {test_auc_pr:.4f}")

    # Calculate mean and standard deviation of test scores across all random states
    mean_test_score = np.mean(test_scores)
    std_test_score = np.std(test_scores)
    print("\n========== Summary of Test Scores ==========")
    print(f"Mean Test AUC-PR Score: {mean_test_score:.4f}")
    print(f"Standard Deviation of Test AUC-PR Score: {std_test_score:.4f}")

    # Return the list of test scores and best models
    return test_scores, best_models, X_test, y_test
    




# Define the preprocessor
ordinal_ftrs = ['Episode_Number']
minmax_ftrs = ['Age']
passthrough_ftrs = ['Sex']

preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(), ordinal_ftrs),
        ('minmax', MinMaxScaler(), minmax_ftrs)],
    remainder='passthrough'  # Pass through the remaining columns untransformed
)

# class_weight = 'balanced'

# Define models and parameter grids
models = {
    'LogisticRegression': (
        LogisticRegression(solver='saga'),  
        {'model__penalty': ['elasticnet'],
         'model__C': [1e-2, 1e-1, 1e0, 1e1],
         'model__l1_ratio': [0, 0.25, 0.5, 0.75, 1]}),
         
    'RandomForestClassifier':(
        RandomForestClassifier(),                         
        {'model__max_depth': [4, 5, 6],
        'model__max_features': [0.65, 0.7, 0.75, 0.8, 0.85]}),

    'KNeighborsClassifier':(
    KNeighborsClassifier(),
    {'model__n_neighbors': [1000, 1500, 1700],
    'model__metric': ['euclidean', 'manhattan'],
    'model__weights': ['uniform']}),

    'XGBClassifier': (
        XGBClassifier(eval_metric='logloss'),
        {"model__reg_alpha": [1e-1, 1e0, 1e1],
        "model__reg_lambda": [ 1e-2, 1e-1, 1e0, 1e1],
        "model__max_depth": [3,5,10,15]}),

    # 'SVC': (
    # SVC(probability=True),
    # {'model__gamma': [1e-1, 1e0, 1e1],
    # 'model__C': [1e-1, 1e0, 1e1]})
}

results = {}

# Loop through each model, run the pipeline, and collect results
for model_name, (model, param_grid) in models.items():
    print(f"\n--- Running model: {model_name} ---")
    test_scores, best_models = MLpipe_KFold_AUC_PR(X, y, preprocessor, model, param_grid)
    mean_auc_pr = np.mean(test_scores)
    std_auc_pr = np.std(test_scores)
    results[model_name] = (mean_auc_pr, std_auc_pr)  # Store results in dictionary
    print(f"\nMean Test AUC-PR Score for {model_name}: {mean_auc_pr:.4f}")
    print(f"Standard Deviation of Test AUC-PR Score for {model_name}: {std_auc_pr:.4f}")


In [ ]:
# from sklearn.metrics import make_scorer, average_precision_score
# from sklearn.model_selection import train_test_split, KFold, GridSearchCV
# from sklearn.pipeline import Pipeline
# import numpy as np
# import joblib

# def MLpipe_KFold_AUC_PR(X, y, preprocessor, ML_algo, param_grid):
#     """
#     Function to perform KFold cross-validation, hyperparameter tuning with GridSearchCV, and evaluate with AUC-PR.
#     Args:
#         X (pd.DataFrame): Feature matrix.
#         y (pd.Series): Target variable.
#         preprocessor (ColumnTransformer): Preprocessor pipeline for data transformation.
#         ML_algo (Estimator): Uninitialized machine learning algorithm (e.g., RandomForestClassifier()).
#         param_grid (dict): Parameter grid for hyperparameter tuning.
#     Returns:
#         best_test_score (float): The highest test AUC-PR score across all random states.
#         best_model (Estimator): The best-performing model with the highest test AUC-PR score.
#         X_test (pd.DataFrame): Test features for later use.
#         y_test (pd.Series): Test targets for later use.
#     """


#     # Initialize variables to track the best model
#     best_test_score = -np.inf  # Negative infinity to ensure any valid score will be higher
#     best_model = None
#     best_random_state = None
#     test_scores = []

#     # Define the AUC-PR scoring function
#     auc_pr_scorer = make_scorer(average_precision_score, needs_proba=True)

#     random_states = [42 * i for i in range(5)]  # Generate random states

#     for random_state in random_states:
#         print(f"\n========== Random State: {random_state} ==========")

#         # Split the data into 'other' (train + validation) and test (80/20 split)
#         X_other, X_test, y_other, y_test = train_test_split(
#             X, y, test_size=0.2, stratify=y, random_state=random_state
#         )

#         # Apply KFold with 4 splits on the 'other' set
#         kf = KFold(n_splits=4, shuffle=True, random_state=random_state)

#         # Create a pipeline with the preprocessor and ML algorithm
#         pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                                     ('model', ML_algo)])
        
#         # Use GridSearchCV for hyperparameter tuning
#         grid_search = GridSearchCV(
#             estimator=pipeline,
#             param_grid=param_grid,
#             cv=kf,
#             scoring=auc_pr_scorer,
#             n_jobs=-1
#         )

#         # Fit the pipeline with GridSearchCV
#         print("Performing GridSearchCV...")
#         grid_search.fit(X_other, y_other)

#         # Print the best parameters found
#         print(f"Best parameters found for Random State {random_state}: {grid_search.best_params_}")

#         # Get the best model from this random state
#         best_model_for_state = grid_search.best_estimator_

#         # Predict probabilities on the test set and calculate AUC-PR
#         y_test_prob = best_model_for_state.predict_proba(X_test)[:, 1]  # Use probabilities for the positive class
#         test_auc_pr = average_precision_score(y_test, y_test_prob)
#         test_scores.append(test_auc_pr)
#         print(f"Test AUC-PR score for Random State {random_state}: {test_auc_pr:.4f}")

#         # Update the best model if the current model performs better
#         if test_auc_pr > best_test_score:
#             best_test_score = test_auc_pr
#             best_model = best_model_for_state
#             best_random_state = random_state


#     # Calculate mean and standard deviation of test scores across all random states
#     mean_test_score = np.mean(test_scores)
#     std_test_score = np.std(test_scores)
#     print("\n========== Summary of Test Scores ==========")
#     print(f"Mean Test AUC-PR Score: {mean_test_score:.4f}")
#     print(f"Standard Deviation of Test AUC-PR Score: {std_test_score:.4f}")


#     print(f"\n========== Best Model Summary ==========")
#     print(f"Best Random State: {best_random_state}")
#     print(f"Best Test AUC-PR Score: {best_test_score:.4f}")


#     # # Save the best model
#     # model_filename = f"best_model_overall.pkl"
#     # joblib.dump(best_model, model_filename)
#     # print(f"Saved best overall model as {model_filename}")

#     # Return the best test score, best model, and test data
#     return best_test_score, test_scores, best_model, X_test, y_test

In [ ]:
from sklearn.metrics import make_scorer, average_precision_score, precision_recall_curve, auc, fbeta_score
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

def MLpipe_KFold_AUC_PR(X, y, preprocessor, ML_algo, param_grid):
    """
    Function to perform KFold cross-validation, hyperparameter tuning with GridSearchCV, and evaluate with AUC-PR.
    Args:
        X (pd.DataFrame): Feature matrix.
        y (pd.Series): Target variable.
        preprocessor (ColumnTransformer): Preprocessor pipeline for data transformation.
        ML_algo (Estimator): Uninitialized machine learning algorithm (e.g., RandomForestClassifier()).
        param_grid (dict): Parameter grid for hyperparameter tuning.
    Returns:
        best_test_score (float): The highest test AUC-PR score across all random states.
        best_model (Estimator): The best-performing model with the highest test AUC-PR score.
        X_test (pd.DataFrame): Test features for later use.
        y_test (pd.Series): Test targets for later use.
    """


    # Initialize variables to track the best model
    best_test_score = -np.inf  # Negative infinity to ensure any valid score will be higher
    best_model = None
    best_random_state = None
    test_scores = []

    # Define the AUC-PR scoring function
    # auc_pr_scorer = make_scorer(average_precision_score, needs_proba=True)

        # Define the scoring function for GridSearchCV
    def auc_pr_scorer(estimator, X_val, y_val):
        y_val_prob = estimator.predict_proba(X_val)[:, 1]  # Probabilities for the positive class
        precision, recall, _ = precision_recall_curve(y_val, y_val_prob)
        return auc(recall, precision)  # Calculate AUC-PR
    

    random_states = [42 * i for i in range(5)]  # Generate random states

    for random_state in random_states:
        print(f"\n========== Random State: {random_state} ==========")

        # Split the data into 'other' (train + validation) and test (80/20 split)
        X_other, X_test, y_other, y_test = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=random_state
        )

        # Apply KFold with 4 splits on the 'other' set
        kf = KFold(n_splits=4, shuffle=True, random_state=random_state)

        # Create a pipeline with the preprocessor and ML algorithm
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('scaler', StandardScaler()),
                                    ('model', ML_algo)
                                    ])
        
        # Use GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            cv=kf,
            scoring=auc_pr_scorer,
            n_jobs=-1
        )

        # Fit the pipeline with GridSearchCV
        print("Performing GridSearchCV...")
        grid_search.fit(X_other, y_other)

        # Print the best parameters found
        print(f"Best parameters found for Random State {random_state}: {grid_search.best_params_}")

        # Get the best model from this random state
        best_model_for_state = grid_search.best_estimator_
        
        # Predict probabilities on the test set and calculate AUC-PR
        y_test_prob = best_model_for_state.predict_proba(X_test)[:, 1]  # Use probabilities for the positive class
        # test_auc_pr = fbeta_score(y_test, y_test_prob, beta=1)
        # test_auc_pr = average_precision_score(y_test, y_test_prob)
        precision, recall, _ = precision_recall_curve(y_test, y_test_prob)
        test_auc_pr = auc(recall, precision)  # Calculate AUC-PR
        test_scores.append(test_auc_pr)
        print(f"Test AUC-PR score for Random State {random_state}: {test_auc_pr:.4f}")

        # Update the best model if the current model performs better
        if test_auc_pr > best_test_score:
            best_test_score = test_auc_pr
            best_model = best_model_for_state
            best_random_state = random_state


    # Calculate mean and standard deviation of test scores across all random states
    mean_test_score = np.mean(test_scores)
    std_test_score = np.std(test_scores)
    print("\n========== Summary of Test Scores ==========")
    print(f"Mean Test AUC-PR Score: {mean_test_score:.4f}")
    print(f"Standard Deviation of Test AUC-PR Score: {std_test_score:.4f}")


    print(f"\n========== Best Model Summary ==========")
    print(f"Best Random State: {best_random_state}")
    print(f"Best Test AUC-PR Score: {best_test_score:.4f}")


    # # Save the best model
    # model_filename = f"best_model_overall.pkl"
    # joblib.dump(best_model, model_filename)
    # print(f"Saved best overall model as {model_filename}")

    # Return the best test score, best model, and test data
    return best_test_score, test_scores, best_model, X_test, y_test


for model_name, (model, param_grid) in models.items():
    print(f"\n--- Running model: {model_name} ---")
    best_test_score, test_scores, best_model, X_test, y_test = MLpipe_KFold_AUC_PR(X, y, preprocessor, model, param_grid)

    # Store the results for the current model
    mean_auc_pr = np.mean(test_scores)
    std_auc_pr = np.std(test_scores)
    results[model_name] = (mean_auc_pr, std_auc_pr)

    # Update the overall best model
    if best_test_score > highest_mean_score:
        highest_mean_score = best_test_score
        best_overall_model = best_model
        best_X_test, best_y_test = X_test, y_test  # Save test data for best overall model
        best_random_state = 42 * np.argmax(test_scores)  # Capture the random state

    # Print results for the current model
    print(f"\nMean Test AUC-PR Score for {model_name}: {mean_auc_pr:.4f}")
    print(f"Standard Deviation of Test AUC-PR Score for {model_name}: {std_auc_pr:.4f}")



# Print sorted results
sorted_results = sorted(results.items(), key=lambda x: x[1][0], reverse=True)
print("\n========== Final Results Sorted by Mean Test AUC-PR Score ==========")
for model_name, (mean_auc_pr, std_auc_pr) in sorted_results:
    print(f"{model_name}: Mean AUC-PR = {mean_auc_pr:.4f}, Std Dev = {std_auc_pr:.4f}")

# Print the best overall model with random state
print("\n========== Best Overall Model ==========")
print(f"Model: {best_overall_model}")
print(f"Highest Test AUC-PR Score: {highest_mean_score:.4f}")
print(f"Corresponding Random State: {best_random_state}")